In [6]:
import pandas as pd
import random
import csv
import os
from datetime import datetime

# Función para mapear las respuestas de texto a valores numéricos (para las otras columnas)
def map_survey_answers(answer):
    if answer == 'Totalmente en desacuerdo':
        return 1
    elif answer == 'En desacuerdo':
        return 2
    elif answer == 'No corresponde':
        return 3
    elif answer == 'De acuerdo':
        return 4
    elif answer == 'Totalmente de acuerdo':
        return 5
    elif answer == 'Masculino':
        return 6  
    elif answer == 'Femenino':
        return 7 
    else:
        return -1  # Valor para respuestas no reconocidas

# Función para determinar una respuesta basada en una probabilidad
def get_answer_based_on_probability(probability):
    if probability >= 90:
        return 5
    elif probability >= 70:
        return 4
    elif probability >= 50:
        return 3
    elif probability >= 30:
        return 2
    else:
        return 1

# Lista para almacenar todas las respuestas
answers = []

# Ruta a la carpeta donde se encuentran los archivos CSV
csv_folder = '../assets/csv/'

# Obtener la lista de archivos CSV en la carpeta
csv_files = [f for f in os.listdir(csv_folder) if f.endswith('.csv')]

# Si hay archivos CSV, obtener el más reciente
if csv_files:
    # Obtener el archivo más reciente
    latest_file = max(
        (os.path.join(csv_folder, f) for f in csv_files),
        key=os.path.getmtime
    )

    # Leer datos reales del archivo CSV más reciente
    data = pd.read_csv(latest_file)

    # Definir la estructura de categorías y pesos
    category_info = {
        'motivational': {
            'columns': ['Mo02', 'Mo08', 'Mo09', 'Mo10', 'Mo11'],
            'weights': [1, 1, 1, 1, 1],
            'min': 5,
            'max': 25
        },
        'health': {
            'columns': ['Ec01', 'Ec03', 'Ec05'],
            'weights': [1, 1, 1],
            'min': 3,
            'max': 15
        },
        'social': {
            'columns': ['Sa01', 'Sa02', 'Sa03', 'Sa04'],
            'weights': [1, 1, 1, 1],
            'min': 4,
            'max': 20
        },
        'familiar': {
            'columns': ['Fa02', 'Fa03', 'Fa07', 'Fa08', 'Fa09'],
            'weights': [1, 1, 1, 1, 1],
            'min': 5,
            'max': 25
        },
        'economy': {
            'columns': ['In02', 'In03', 'In04', 'In05'],
            'weights': [1, 1, 1, 1],
            'min': 4,
            'max': 20
        }
    }



    # Iterar sobre cada fila de datos
    for i in range(data.shape[0]):
        row = []

        # Otras columnas: Respuestas a las preguntas
        for j in range(1, data.shape[1]):
            row.append(map_survey_answers(data.iloc[i, j]))
        
        answers.append(row)

    # Verificar si alguna fila tiene un valor incorrecto para 'Sex'
    for ans in answers:
        if ans[0] not in [6, 7]:  # Si el valor de 'Sex' no es 6 o 7, muestra la fila

         print(len(answers))  # Imprimir el número total de respuestas

    # Calcular las probabilidades y las intenciones
    for row in answers:
        motivational_probability = (sum(row[data.columns.get_loc(col)] * category_info['motivational']['weights'][idx] 
                                          for idx, col in enumerate(category_info['motivational']['columns'])) 
                                    - category_info['motivational']['min']) / (category_info['motivational']['max'] - category_info['motivational']['min']) * 100

        health_probability = (sum(row[data.columns.get_loc(col)] * category_info['health']['weights'][idx] 
                                  for idx, col in enumerate(category_info['health']['columns'])) 
                              - category_info['health']['min']) / (category_info['health']['max'] - category_info['health']['min']) * 100

        social_probability = (sum(row[data.columns.get_loc(col)] * category_info['social']['weights'][idx] 
                                  for idx, col in enumerate(category_info['social']['columns'])) 
                              - category_info['social']['min']) / (category_info['social']['max'] - category_info['social']['min']) * 100
        
        # Calcular intención basada en las probabilidades
        intention = [
            get_answer_based_on_probability((0.228 * health_probability + 0.116 * social_probability) / 0.344),
            get_answer_based_on_probability(health_probability),
            get_answer_based_on_probability((0.1 * health_probability + 0.121 * social_probability) / 0.221),
            get_answer_based_on_probability(motivational_probability)
        ]

        # Agregar la intención a la fila de respuestas
        row.extend(intention)

    # Definir los encabezados para el archivo CSV de salida
    headers = ['Sex', 'Ac02', 'Ac03', 'Ac06', 'Ac10', 'Ac13', 'Ec01', 'Ec03', 'Ec05', 'Fa02', 'Fa03', 'Fa07', 
               'Fa08', 'Fa09', 'In02', 'In03', 'In04', 'In05', 'Mo02', 'Mo08', 'Mo09', 'Mo10', 'Mo11', 
               'Sa01', 'Sa02', 'Sa03', 'Sa04', 'So01', 'So04', 'So07', 'So08', 'Intention1', 'Intention2', 
               'Intention3', 'Intention4']

    # Crear la carpeta 'answers' si no existe
    output_folder = 'answers'
    os.makedirs(output_folder, exist_ok=True)

    # Crear el nombre del archivo de salida con la fecha y hora actuales
    base_filename = f'answers_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.csv'
    output_file = os.path.join(output_folder, base_filename)

    # Verificar si el archivo ya existe y generar un nuevo nombre si es necesario
    counter = 1
    while os.path.exists(output_file):
        output_file = os.path.join(output_folder, f'answers_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}_{counter}.csv')
        counter += 1

    # Escribir las respuestas en el archivo CSV
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)

        writer.writerow(headers)
        writer.writerows(answers)
else:
    print("No se encontraron archivos CSV en la carpeta especificada.")


Social columns: ['Sa01', 'Sa02', 'Sa03', 'Sa04']
Social weights: [1, 1, 1, 1]
1056
